# Lecture 6

The **objective** of this lecture is to find _similar_ cities based on their emissions profile.  It is, in effect, an introduction to clustering.  

The real-world relevance of this work is that mayors talk to other mayors. The number of conferences where the mayor of Carmel, California has spoken to the mayor of Des Moines, Iowa is insane, surprising. Suppose there is a conference of mayors for climate mitigation. Who should the mayor of Carmel talk to about policy strategies? What policy strategies have worked in _similar_ cities?  What are the _similar_ cities?

In [1]:
import pandas
import matplotlib.pyplot as plt

In [48]:
df = pandas.read_csv("wide_2015.csv")
sector_emissions = df.iloc[:, 4:]
sector_emissions = df[["onroad", "residential", "nonroad"]]

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [49]:
X = sector_emissions.values
model = KMeans(n_clusters=10)
model.fit(X)

# Prediction on the entire data
all_predictions = model.predict(X)
all_predictions

import numpy
numpy.histogram(all_predictions, bins=4)

(array([3284,   65,    9,  243]), array([0.  , 2.25, 4.5 , 6.75, 9.  ]))